In [ ]:
from processing_arithmetics.arithmetics.treebanks import treebank
from processing_arithmetics.arithmetics.MathExpression import MathExpression
from processing_arithmetics.arithmetics.MathTreebank import MathTreebank
from processing_arithmetics.sequential.architectures import DiagnosticClassifier
import numpy as np
import pandas as pd
from plotnine import *
import plotnine


In [ ]:
basename = "2018-10-11T11+02:00"
digits = np.arange(-10,11)
operators = ['+', '-']
#classifiers = "subtracting intermediate_locally intermediate_recursively grammatical depth minus1depth minus2depth minus3depth minus4depth minus1depth_count switch_mode".split()
classifiers = "intermediate_locally intermediate_recursively".split()
arch1 = DiagnosticClassifier(digits=digits,
                             operators=operators,
                             classifiers=classifiers)
arch1.add_pretrained_model(model="./diagnoses/ScalarPrediction_GRU_infix_{}_10_dc8.h5".format(basename))
arch1.model.compile(loss=arch1.loss_functions, 
                    optimizer='adam',
                    metrics=arch1.metrics,
                    loss_weights=arch1.loss_weights)

arch2 = DiagnosticClassifier(digits=digits,
                         operators=operators,
                         classifiers=classifiers)
arch2.add_pretrained_model(model="./diagnoses/ScalarPrediction_GRU_infix_{}_16_dc8.h5".format(basename))
arch2.model.compile(loss=arch2.loss_functions,
                    optimizer='adam',
                    metrics=arch2.metrics,
                    loss_weights=arch2.loss_weights)


In [ ]:
#tb = treebank(seed=0, kind='test', debug=True)
#tb = dict(tb)
#tb = MathTreebank({L:5 for L in ['L1','L2','L3', 'L4','L5','L6','L7', 'L8', 'L9']}, digits=digits)
tb = MathTreebank({L:5 for L in ['L3']}, digits=digits)

In [ ]:
for example in tb.examples:
    display(example[0])

In [ ]:
#tb.examples = []
#tb.add_example_from_string('( 7 - ( 4 - 3 )  )')
data = arch1.generate_test_data(tb, digits=digits)
results = {}
for name, X, Y in data:
    results[name] = {}
    for n, arch in [("10", arch1), ("16", arch2)]:
        result = np.array(arch.model.predict(X))
        result = result.reshape(*result.shape[:-1])
        results[name][n] = result
        #print "MODEL: ", n
        #print "\texpected:"
        #for k,v in Y.items():
        #    print "\t\t", k, v.shape
        #print "\tresult:", result.shape

In [ ]:
results = results['test treebank']

In [ ]:
plot_data = []
for model in results:
    model_results = results[model]
    #print model, model_results.shape
    for n, example in enumerate(tb.examples):
        #display(example[0])
        seq_len = len(list(example[0].iterate('infix')))
        for i, classifier in enumerate(classifiers):      
            #print i, classifier
            #print model_results[i, n, -seq_len:]
            #print data[0][2][classifier][n,-seq_len:,0]
            plot_data.append(pd.DataFrame({
                'model': model,
                'example': n,
                'classifier': classifier,
                'prediction': model_results[i, n, -seq_len:],
                'expected': data[0][2][classifier][n,-seq_len:,0],
            }).reset_index())
plot_df = pd.concat(plot_data).reset_index(drop=True)

In [ ]:
plot_df = plot_df.melt(id_vars=['index', 'classifier', 'example', 'model'], value_vars=['expected', 'prediction'])
plot_df = plot_df[plot_df['model'] == "10"]
display(plot_df)

In [ ]:
plotnine.options.figure_size = (12,len(tb.examples) * 2)
ggplot(plot_df, aes(x="index", y="value")) + \
    geom_step(aes(linetype="variable")) + \
    facet_grid("example~classifier")